# Multi-Modal Logistics Parks - India
** Problem from: IGSA UW-Madison SUPPLY CHAIN HACKATHON - Apr2020** 
### Solution author : Adhokshaja Achar Budihal Prasad

## Problem Statement
Development of Multi-Modal Logistics Parks (MMLP) at strategic locations to enable efficient inter-modal (road, rail, waterway and air) freight movement and formalised interface amongst logistics service providers, service users and regulators is expected to bring about significant efficiencies in India’s logistics sector, providing a much-needed impetus towards driving logistics cost to less than 10% of country’s GDP. To achieve that goal, the government has embarked on ambitious journey to develop a network of MMLPs across the country. The complexities involved in its implementation, criticality of the infrastructure and need for smooth operations involving multiple stakeholders necessitates strong focus from the government on its development and operations. This becomes more important especially in the current unprecedented challenging times of COVID-19.

In this question, we would solve a Minimum-cost flow problem, which is an optimization problem to find the cheapest possible way of sending a certain amount of flow through a flow network. The flow network is the above described network of ports to possible MMLP locations with the nodes being either source or demand nodes and the edges being the distance between nodes. The Data File is a dense matrix where the rows index are the ports and the column index are the demand zones to be satisfied. The matrix itself is the distance between these locations in kilometer (km). The cost per kilometer is 35 Rs. Find the minimum cost to satisfy the demand. 


## Author's notes
- This has been adapted to python from the Julia Jupyter Notebook files provided.
- The names for cities/ports in the original distance csv were not consistant with the city/port in the supply demand excel file. These have been corrected to be made consistent before import
- In addition to `pandas` this solution uses `pyomo` with `glpk` solver. These can be installed in a Conda environment by using
 ```
 conda install -c conda-forge glpk
 conda install -c conda-forge pyomo
 ```


In [1]:
import pandas as pd
from pyomo.opt import SolverFactory
from pyomo.environ import *

## Define ports and Cities
Ports are origin, Cities are destination

In [15]:
Ports = ["Kolkata", "Haldia", "Paradeep", "Vizag", "Chennai", "Chidambaram", "Kochi", "Mangalore", "Mormugaon", "JNPT", "DeenDayal", "Kharagpur", "Chandigarh", "Delhi", "Bangalore"]
Cities = [ "Guwhathi",  "Srinagar",  "Chandigarh", "Delhi", "Rudrapur","NaviMumbai","Vijaywada","Varanasi","Kharagpur", "Indore", "Belgaum","Bangalore", "Rajkot", "Ahmedabad", "Ludhiana", "Rourkela", "Hyderabad","Nagpur", "Coimbatore", "Kolkata"]
len(Cities)

20

## Read inputs from data file
Original Julia code coalesced null values with 2000 - Do the same here 

_Note: The names for cities/ports in the original distance csv were not consistant with the city/port in the supply demand excel file. These have been modified to be made consistent before import_ 

In [3]:

distances_df = pd.read_csv("Data.csv", index_col=0).fillna(2000)

# Coellesc null values with 2000 ( from the original file)

distancesDict = dict()
## the dictionary has port,city tupes with the value as the distance between them

for p in Ports:
   for c in Cities:
      distancesDict[(p,c)] = distances_df.loc[p,c]
      

## Read Supply and demand data

In [4]:
#read demand sypply data

demandsupply_df = pd.read_excel("Supply Demand Data.xlsx")

# Convert supply and demand to dictionary
supply = demandsupply_df[['Supply','Unnamed: 1']].dropna().set_index('Supply').to_dict()['Unnamed: 1']
demand = demandsupply_df[['Demand', 'Unnamed: 4']].dropna().set_index('Demand').to_dict()['Unnamed: 4']

unit_cost = 35 # Rs./km

# Solution: GLPK model

The objective here is to minimize the cost associated with transport from ports to cities. This is calculated by multiplying the units shipped with the distance and the cost per km


In [5]:

model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Set inputs ( these are raw data being inputted)
model.ports = Set(initialize=supply.keys(), doc='Ports')
model.cities = Set(initialize=demand.keys(), doc='Cities')


model.distance = Param(model.ports, model.cities, initialize=distancesDict, doc='Distance in km')
model.unitCost = Param(initialize=unit_cost, doc='Freight cost in Rs.per km')

# Function to calculate cost - Distance * unit cost
def perUnitCost_init(model, ports, cities):
  return model.unitCost * model.distance[ports,cities]

model.freightCost = Param(model.ports, model.cities, initialize=perUnitCost_init, doc='Transport cost in Rs.')


# Initialize routes - Flow : number of pieces to be moved
model.flow = Var(model.ports, model.cities, bounds=(0.0,None),domain = NonNegativeReals, doc='Flow quantities')


# Function to get cost of moving freight - Objective function to be minimized
def costCalc(model):
  return sum(model.freightCost[port,city]*model.flow[port,city] for port in model.ports for city in model.cities)

# add objective to model
model.cost = Objective(rule=costCalc, sense=minimize, doc='Total cost')


# define Supply constraint
def supplyConstraint(model, port):
  return sum(model.flow[port,city] for city in model.cities) <= supply[port]

# add supply constraint to model
model.supply_cons = Constraint(model.ports, rule=supplyConstraint, doc='Observe supply limit at port i')

# define demand constraint
def demandConstraint(model, city):
  return sum(model.flow[port,city] for port in model.ports) >= demand[city]  

# add demand constraint to model
model.demand_cons = Constraint(model.cities, rule=demandConstraint, doc='Satisfy demand at city c')

# Save LP file
model.write('AABP-PortCityModel.lp')




('AABP-PortCityModel.lp', 3129222512824)

In [6]:
# solve using glpk solver
results = SolverFactory("glpk").solve(model)
#  results.write()

In [7]:

res_arr = [] # 2-d array to be converted to df later
if 'ok' == str(results.Solver.status):
    totalCost = model.cost()
    for port in model.ports:
        for city in model.cities:
            if model.flow[port,city]() > 0:
                res_arr.append((port,city,model.flow[port,city](),model.freightCost[port,city]/1000, model.freightCost[port,city]/70000))           
    result_df = pd.DataFrame(res_arr, columns=['Port','City','Flow','Cost (Rs. in thousands)', 'Cost ($ in thousands)'])
else:
    print("Couldn't arrive on solution")

## Solution results
### Flow quantities
Flow is from port to city

In [8]:
result_df.to_csv('./soltion.csv',index=False)
result_df

,Port,City,Flow,Cost (Rs. in thousands),Cost ($ in thousands)
0,Kolkata,Kolkata,31.6,0.000,0.0000
1,Haldia,Kolkata,51.0,4.165,0.0595
2,Paradeep,Guwhathi,26.0,49.840,0.7120
3,Paradeep,Srinagar,32.1,62.545,0.8935
4,Paradeep,Delhi,9.1,60.165,0.8595
5,Paradeep,Rudrapur,38.0,3.465,0.0495
6,Paradeep,Varanasi,32.6,30.730,0.4390
7,Paradeep,Kharagpur,21.7,12.180,0.1740
8,Paradeep,Rourkela,40.2,13.790,0.1970
9,Paradeep,Kolkata,39.3,15.785,0.2255


### Optimized cost

In [9]:
print("Total cost will be Rs.",model.cost()/100000, "(lakhs)")
print("Total cost will be $", model.cost()/70000,"(thousands)")

Total cost will be Rs. 446.847835 (lakhs)
Total cost will be $ 638.35405 (thousands)
